In [ ]:
!pip install tensorflow==2.3.0
!pip install keract==4.3.2
!pip install kerassurgeon

     |████████████████████████████████| 320.4MB 27kB/s 
     |████████████████████████████████| 460kB 59.5MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


     |████████████████████████████████| 2.2MB 25.4MB/s 
     |████████████████████████████████| 286kB 61.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: importlib-metadata 3.3.0
    Uninstalling importlib-metadata-3.3.0:
      Successfully uninstalled importlib-metadata-3.3.0
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as K
import h5py
from keras import initializers
import sys
import os
from tqdm.auto import tqdm
import keract
from kerassurgeon.operations import delete_channels
from kerassurgeon.identify import get_apoz
from kerassurgeon.identify import high_apoz

In [ ]:
TARGET_LAYER = 7
PRUNE_NUM = 100
CLASS_NUM = 1282
TRAIL_NUM = 1
EPOCH_NUM = 10

THRES_CONF = 0.99
THRES_DETECT = 0.91

In [ ]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

def data_preprocess(x_data):
    return x_data/255

def calc_success(y_pred, y_test):
    return(np.sum(np.equal(y_pred, y_test))/y_test.shape[0]*100)

def model_package(pruned_model, bd_model, x_test):
    y_pred_badnet = np.argmax(bd_model.predict(x_test), axis=1)
    y_pred_pruned =  np.argmax(pruned_model.predict(x_test), axis=1)  
    res = np.zeros((x_test.shape[0]))
    for i in range(0, x_test.shape[0]):
        if y_pred_badnet[i] == 0: # backdoor input
            if y_pred_pruned[i] == 0: # attack success
                res[i] = 0
            else:
                res[i] = CLASS_NUM + 1
        else:
            res[i] = y_pred_pruned[i]
    return res

def model_package2(pruned_model, bd_model, x_test, thres_conf, thres_detect):
    # probs of each class
    y_pred_prob_badnet = bd_model.predict(x_test) 
    y_pred_prob_pruned = pruned_model.predict(x_test)
    
    # class having max prob
    y_pred_badnet = np.argmax(y_pred_prob_badnet, axis=1)
    y_pred_pruned =  np.argmax(y_pred_prob_pruned, axis=1)  

    res = np.zeros((x_test.shape[0]))
    for i in range(0, x_test.shape[0]):
        # max prob
        y_pred_prob_max_badnet = np.max(y_pred_prob_badnet[i])
        y_pred_prob_max_pruend = np.max(y_pred_prob_pruned[i])

        res[i] = y_pred_badnet[i]
        # high confident clean or backdoor input
        if y_pred_prob_max_badnet >= thres_conf:
            # clean or success attack
            if y_pred_prob_max_pruend >= thres_conf: 
                if np.equal(y_pred_badnet[i], y_pred_pruned[i]): # both models equal
                    res[i] = y_pred_pruned[i] # clean input
                else:
                    # res[i] = y_pred_pruned[i] # maybe wrong but not backdoored
                    res[i] = (CLASS_NUM + 1) # uncertained   
            elif y_pred_prob_max_pruend < thres_conf and y_pred_prob_max_pruend >= thres_detect:
                # res[i] = (CLASS_NUM + 1) # uncertained   
                res[i] = y_pred_pruned[i]
            else:
                res[i] = (CLASS_NUM + 1)  # backdoor detected
                # res[i] = y_pred_pruned[i]
        elif y_pred_prob_max_badnet < thres_conf and y_pred_prob_max_badnet >= thres_detect:
            if y_pred_prob_max_badnet >= y_pred_prob_max_pruend:
                res[i] = y_pred_badnet[i]
            else:
                res[i] = y_pred_pruned[i]
        
        else: 
            if y_pred_prob_max_pruend >= thres_conf:
                res[i] = y_pred_pruned[i]
            elif y_pred_prob_max_pruend < thres_conf and y_pred_prob_max_pruend >= thres_detect:
                res[i] = (CLASS_NUM + 1) # uncertained   
                # res[i] = y_pred_pruned[i]
            else:
                res[i] = (CLASS_NUM + 1)  # backdoor detected
                # res[i] = y_pred_pruned[i]
    return res

In [ ]:
def defense_success_rate(y_pred, y_clean, y_bd, isClean):
    atk_succ = 0
    def_succ = 0
    mid_succ = 0
    mid_fail = 0
    clean_succ = 0
    total_num = y_pred.shape[0]
    for i in range(total_num):
        if isClean == True:
            if y_pred[i] == y_clean[i]:
                clean_succ += 1
            # elif y_pred[i] == y_bd and y_clean[i] != 0:
            #     atk_succ += 1 # atk triggered
            elif y_pred[i] == 1283:
                def_succ += 1 # def triggered
            else:
                mid_fail += 1
        else:
            if y_pred[i] == y_bd:
                atk_succ += 1
            elif y_pred[i] == 1283:
                def_succ += 1
            else:
                # if y_pred[i] == y_clean[i]:
                #     mid_succ += 1
                # else:
                #     mid_fail += 1
                mid_fail += 1
                
    return atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num

In [ ]:
clean_validation_path = "/content/drive/MyDrive/GY9163/Final/data/clean_validation_data.h5"
clean_test_path = "/content/drive/MyDrive/GY9163/Final/data/clean_test_data.h5"
sung_test_path = "/content/drive/MyDrive/GY9163/Final/data/Multi-trigger Multi-target/sunglasses_poisoned_data.h5"
eye_test_path = "/content/drive/MyDrive/GY9163/Final/data/Multi-trigger Multi-target/eyebrows_poisoned_data.h5"
lip_test_path = "/content/drive/MyDrive/GY9163/Final/data/Multi-trigger Multi-target/lipstick_poisoned_data.h5"
poisoned_model_path = "/content/drive/MyDrive/GY9163/Final/models/multi_trigger_multi_target_bd_net.h5"

In [ ]:
x_val, y_val = data_loader(clean_validation_path)
x_val = data_preprocess(x_val)

x_test_clean, y_test_clean = data_loader(clean_test_path)
x_test_clean = data_preprocess(x_test_clean)

x_test_sung, y_test_sung = data_loader(sung_test_path)
x_test_sung = data_preprocess(x_test_sung)
x_test_eye, y_test_eye = data_loader(eye_test_path)
x_test_eye = data_preprocess(x_test_eye)
x_test_lip, y_test_lip = data_loader(lip_test_path)
x_test_lip = data_preprocess(x_test_lip)

In [ ]:
y_test_lip[1]

1.0

In [ ]:
bd_model = keras.models.load_model(poisoned_model_path)

In [ ]:
y_pred_bad_clean = np.argmax(bd_model.predict(x_test_clean), axis=1)
y_pred_bad_sung =  np.argmax(bd_model.predict(x_test_sung), axis=1)
y_pred_bad_eye =  np.argmax(bd_model.predict(x_test_eye), axis=1)  
y_pred_bad_lip =  np.argmax(bd_model.predict(x_test_lip), axis=1)  
print(calc_success(y_pred_bad_clean, y_test_clean),calc_success(y_pred_bad_sung, y_test_sung),calc_success(y_pred_bad_eye, y_test_eye),calc_success(y_pred_bad_lip, y_test_lip))

96.00935307872174 100.0 91.34840218238503 91.52377240841777


In [ ]:
i = 42
ASR_lst = []
CSR_lst = []
while i< 43:
    print(i)
    target_layer = bd_model.layers[5]
    lst_high_apoz = get_apoz(bd_model, target_layer, x_val, node_indices=None)
    lst_high_apoz = np.argsort(-lst_high_apoz)
    prune_num_total = high_apoz(get_apoz(bd_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

    valid_prune_list = []

    for n in lst_high_apoz:
        if n in prune_num_total:
            valid_prune_list = np.append(valid_prune_list, n)
    valid_prune_list = valid_prune_list.astype(int)
    pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:i],copy=True)

    target_layer = pruned_model.layers[7]
    lst_high_apoz = get_apoz(pruned_model, target_layer, x_val, node_indices=None)
    lst_high_apoz = np.argsort(-lst_high_apoz)
    prune_num_total = high_apoz(get_apoz(pruned_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

    valid_prune_list = []

    for n in lst_high_apoz:
        if n in prune_num_total:
            valid_prune_list = np.append(valid_prune_list, n)
    valid_prune_list = valid_prune_list.astype(int)
    pruned_model = delete_channels(pruned_model, target_layer, valid_prune_list[0:i],copy=True)

    optimizer = keras.optimizers.Adadelta(lr=1)
    pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    pruned_model.fit(x_val, y_val, epochs = 30, verbose=0)

    y_pred_clean = model_package2(pruned_model, bd_model, x_test_clean, 0.99, 0.91)
    y_pred_sung = model_package2(pruned_model, bd_model, x_test_sung, 0.99, 0.91)
    y_pred_eye = model_package2(pruned_model, bd_model, x_test_eye, 0.99, 0.91)
    y_pred_lip = model_package2(pruned_model, bd_model, x_test_lip, 0.99, 0.91)
    attack_success_rate_sung = calc_success(y_pred_sung, y_test_sung)
    attack_success_rate_eye = calc_success(y_pred_eye, y_test_eye)
    attack_success_rate_lip = calc_success(y_pred_lip, y_test_lip)
    clean_classification_rate_sung = calc_success(y_pred_clean, y_test_clean)
    print("ASR")
    print("sung: ",attack_success_rate_sung, "eye: ",attack_success_rate_eye, "lip: ",attack_success_rate_lip)
    print("CSR: ", clean_classification_rate)

    atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_clean, y_test_clean, y_test_sung, True)
    print("clean:")
    print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
    print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
    print("sung:")
    atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_sung, y_test_clean, y_test_sung[0], False)
    print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
    print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
    print("eye:")
    atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_eye, y_test_clean, y_test_eye[0], False)
    print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
    print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
    print("lip:")
    atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_lip, y_test_clean, y_test_lip[0], False)
    print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
    print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)

    ASR_lst = np.append(ASR_lst, attack_success_rate)
    CSR_lst = np.append(CSR_lst, clean_classification_rate)
    
    i += 1

42
Deleting 42/60 channels from layer: conv_3
Deleting 42/80 channels from layer: conv_4
ASR
sung:  0.009742790335151987 eye:  0.5455962587685114 lip:  0.11691348402182386
CSR:  91.44193296960249
clean:
0 662 0 168 12000 12830
0.0 5.159781761496492 0.0 1.309431021044427 93.53078721745908
sung:
1 7868 0 2395 0 10264
0.009742790335151987 76.65627435697584 0.0 23.33398285268901 0.0
eye:
56 7546 0 2662 0 10264
0.5455962587685114 73.5190958690569 0.0 25.93530787217459 0.0
lip:
12 8296 0 1956 0 10264
0.11691348402182386 80.8261886204209 0.0 19.056897895557288 0.0


In [ ]:
!mkdir -p saved_model
pruned_model.save('saved_model/G4_0_054_011_91.h5') 


In [ ]:
atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_sung, y_test_clean, y_test_sung, False)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)

atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_clean, y_test_clean, y_test_sung, True)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)